<a href="https://colab.research.google.com/github/HyungunKim/IMLAB_SelfDrivingCarla/blob/main/experiments/FKM_video_JH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/class_home/SF2021/Forward\ Kinematics\ Model

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1QnAn1UfhSxAGeRHXouSfr-_KmpnWU4dt/class_home/SF2021/Forward Kinematics Model


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import math
import os 
import csv
import shutil
import matplotlib.pyplot as plt
# Enable interactive plot
%matplotlib notebook

In [4]:
file_name = f'./vlog_v2/vlog_1.csv'
df = pd.read_csv(file_name)
min_x = df['x'].min()
min_y = df['y'].min()
min_yaw = df['yaw'].min()
min_vx = df['vx'].min()
min_vy = df['vy'].min()
min_spds = math.sqrt(min_vx**2 + min_vy**2)

max_x = df['x'].max()
max_y = df['y'].max()
max_yaw = df['yaw'].max()
max_vx = df['vx'].max()
max_vy = df['vy'].max()
max_spds = math.sqrt(max_vx**2 + max_vy**2)

In [5]:
REAR_WB = 1.3851432
FRONT_WB = -0.73497885     # fix
THROT_ACCEL = 3.5279548  # fix
BRAKE_ACCEL = 11.411816
STEER_GAIN = 0.2029629  # fix
AERO_RESIST = 0.044634644
ROLL_RESIST = 0.839375
GRAV_RESIST = -2.2889855

In [6]:
def make_train_data(file_name):
    df = pd.read_csv(file_name)
    
    time_index = np.round(df['t'],2)
    df['yaw'] = df['yaw'] * 3.14159265359/180   # 180 -> radian

    npvx = df['vx'].values                      # calculate val
    npvy = df['vy'].values
    npspds = np.sqrt(npvx**2 + npvy**2)
    df['spds'] = npspds
    
    # delete unnecessary data
    df.pop('z')
    df.pop('vz')
    df.pop('roll')
    df.pop('reverse')
    df['spds'] = npspds
    
    # necessary data
    x_df_locs = df[['x', 'y']]
    x_df_yaws = df[['yaw']]
    x_df_spds = df[['spds']]
    x_df_pich = df[['pitch']]
    x_df_acts = df[['steer','throttle','brake']]

    xlocs = x_df_locs.values
    xyaws = x_df_yaws.values
    xspds = x_df_spds.values
    xpich = x_df_pich.values
    xacts = x_df_acts.values
    

    X = np.hstack([xlocs, xyaws, xspds, xpich, xacts])

    return X, time_index

In [7]:
def TimeTest(X, Act_np, time):
    dt = 0.1 # time step
    idx = int(time*10)
    
    X_locs = X[:2]
    X_yaws = X[2:3]
    X_spds = X[3:4]
    
    
    X_acts = Act_np[idx][4:]
    
    X_pich = X_acts[0:1]
    steer = X_acts[1:2]
    throt = X_acts[2:3]
    brake = X_acts[3:4] #.byte()

    if (brake == 0):
        accel = THROT_ACCEL*throt
    else:
        accel = -BRAKE_ACCEL
    accel -= AERO_RESIST * X_spds + ROLL_RESIST # + GRAV_RESIST * X_pich
    
    wheel = STEER_GAIN * steer
    beta = np.arctan(REAR_WB * np.tan(wheel)/(FRONT_WB + REAR_WB)) #라디안 값
        
    next_locs = X_locs + (X_spds * (np.concatenate([np.cos(X_yaws+beta), np.sin(X_yaws+beta)], -1) * dt))
    # next_yaws = yaws + divide(spds* sin(beta) * self.dt, self.rear_wb  + 0.001) # rad
    next_yaws = X_yaws + (X_spds * (np.sin(beta) * dt))/(REAR_WB)
    next_spds = X_spds + accel * dt
    # y = next_locs, next_yaws, next_speeds
    
    
    if next_yaws > np.pi:
        next_yaws -= 2 * np.pi
    elif next_yaws < -np.pi:
        next_yaws += 2 * np.pi
    
    return np.concatenate([next_locs, next_yaws, np.maximum(np.zeros_like(next_spds), next_spds)], -1)

    

In [8]:
dt = 0.1
future_time = 5
future = int(future_time / dt )

test_time5 = np.zeros((5,10000-future,future,4))  # file_number, time_index, time_step, feature
X_data_list = []

for i in range(1,6,1):
    file_name = f'./vlog_v2/vlog_{i}.csv'
    x, time_index = make_train_data(file_name)    # x - (x, y, yaw, spds, pitch, steer, throttle, brake)
    X_data_list.append(x)
    
    for j in range(len(time_index)-future):
        time = time_index[j]    
        next_x = x[j]
        
        next_x = next_x[:4]
        
        for k in range(future):
            for p in range(4):
                test_time5[i-1][j][k][p] = next_x[p]  
            next_x = TimeTest(next_x, x, time+k*0.1)
        
    print(test_time5.shape)
    
    

(5, 9950, 50, 4)
(5, 9950, 50, 4)
(5, 9950, 50, 4)
(5, 9950, 50, 4)
(5, 9950, 50, 4)


In [9]:
%matplotlib notebook
def show_subplot(test_time, X_data_list,i,k):
    for j in range(4):
        
        t1 = np.arange(0,50,1)
        t2 = np.arange(0,50,1)
        # plt.figure()
        plt.subplot(2,2,j+1)
        plt.plot(t1, test_time[k][i][:,j], 'r-', t2, X_data_list[k][i:(i+50)][:,:4][:,j])
        # red - predict, blue - ground truth
        plt.show()
i = 1000
k = 1
show_subplot(test_time5 ,X_data_list, i, k)
plt.show()

# i = 830
# k = 0
# show_subplot(test_time5 ,X_data_list, i, k)
# plt.show()


<IPython.core.display.Javascript object>

In [10]:
testtime0 = test_time5[0,:,:,0]
Ytime0 = np.squeeze(X_data_list[0][:,:1])
testtime1 = test_time5[0,:,:,1]
Ytime1 = np.squeeze(X_data_list[0][:,1:2])
testtime2 = test_time5[0,:,:,2]
Ytime2 = np.squeeze(X_data_list[0][:,2:3])
testtime3 = test_time5[0,:,:,3]
Ytime3 = np.squeeze(X_data_list[0][:,3:4])


In [11]:
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np
import random
import time

%matplotlib notebook

# location units for each plot
left = -5
right = 50
max_x += 10
max_y += 10
top1 = min_x + (max_x - min_x) * 1.05
bottom1 = min_x - (max_x - min_x) * 0.1
top2 = min_y + (max_y - min_y) * 1.05
bottom2 = min_y - (max_y - min_y) * 0.1
top3 = 4 + 0.3
bottom3 = -4.7
top4 = 10.5 + 0.5
bottom4 = -1

# xtickx time_interval (0,1,2,3,4,5)
time_idx = np.array([0,9,19,29,39,49])
time_label = time_idx // 10 + 1
time_label[0] = 0

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2)

plt.subplots_adjust(left=0.125, bottom=0.1, right = 0.9, top=0.9, wspace=0.3, hspace=0.35)

ax1.set_xlim(0, 50)
ax1.set_ylim(min_x, max_x)
ax1.title.set_text('X')
ax1.text(left, top1, '(m)')
ax1.text(right, bottom1, '(s)')
ax1.set_xticks(time_idx)
ax1.set_xticklabels([time_label[0],time_label[1],time_label[2],time_label[3],time_label[4],time_label[5]])

ax2.set_xlim(0, 50)
ax2.set_ylim(min_y, max_y)
ax2.title.set_text('Y')
ax2.text(left, top2, '(m)')
ax2.text(right, bottom2, '(s)')
ax2.set_xticks(time_idx)
ax2.set_xticklabels([time_label[0],time_label[1],time_label[2],time_label[3],time_label[4],time_label[5]])

ax3.set_xlim(0, 50)
ax3.set_ylim(-4, 4)
ax3.title.set_text('yaw')
ax3.text(left, top3, '(rad)')
ax3.text(right, bottom3, '(s)')
ax3.set_xticks(time_idx)
ax3.set_xticklabels([time_label[0],time_label[1],time_label[2],time_label[3],time_label[4],time_label[5]])

ax4.set_xlim(0, 50)
ax4.set_ylim(0, 10.5)
ax4.title.set_text('speeds')
ax4.text(left, top4, '(m/s)')
ax4.text(right, bottom4, '(s)')
ax4.set_xticks(time_idx)
ax4.set_xticklabels([time_label[0],time_label[1],time_label[2],time_label[3],time_label[4],time_label[5]])


x = np.arange(0,50)
line1, = ax1.plot([], [], color = 'r')
line2, = ax1.plot([], [], lw=0.5, color='b')
line3, = ax2.plot([], [], color = 'r')
line4, = ax2.plot([], [], lw=0.5, color='b')
line5, = ax3.plot([], [], color = 'r')
line6, = ax3.plot([], [], lw=0.5, color='b')
line7, = ax4.plot([], [], color = 'r')
line8, = ax4.plot([], [], lw=0.5, color='b')

ax2.legend([line3, line4], ['pred', 'truth'], bbox_to_anchor=(1.3,1.3))
line = [line1, line2, line3, line4, line5, line6, line7, line8]

def init():
    return line

x = np.linspace(0,5,101)
# print(x)
def animate1(i):
    x = np.arange(0,50)
    y = testtime0[i,:]
    y2 = Ytime0[i:i+50]
    y3 = testtime1[i,:]
    y4 = Ytime1[i:i+50]
    y5 = testtime2[i,:]
    y6 = Ytime2[i:i+50]
    y7 = testtime3[i,:]
    y8 = Ytime3[i:i+50]
    
    line[0].set_data(x,y)
    line[1].set_data(x,y2)
    line[2].set_data(x,y3)
    line[3].set_data(x,y4)
    line[4].set_data(x,y5)
    line[5].set_data(x,y6)
    line[6].set_data(x,y7)
    line[7].set_data(x,y8)
    return line


anim = animation.FuncAnimation(fig, animate1, init_func= init ,frames=9900, interval=10, blit=False, repeat=False)
#anim.save(r'animation_5.gif', fps=200)

<IPython.core.display.Javascript object>

In [12]:
HTML_VIDEO = anim.to_html5_video()

In [13]:
from IPython.core.display import display, HTML
display(HTML(HTML_VIDEO))